In [5]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 2.4 MB/s eta 0:00:000:00:010:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.0/575.0 KB 6.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 22.5 MB/s eta 0:00:00m eta 0:00:010:01:01
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached aiosignal-1.2.0-py3-none-any.whl (8.2 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 KB 4.8 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
  A

In [6]:
import torch

In [7]:
torch.cuda.is_available()

False

In [10]:
import os
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [9]:
with open('./ParaSCI/Data/ParaSCI-ACL/train/train.src') as file:
    train_source = file.readlines()

with open('./ParaSCI/Data/ParaSCI-ACL/train/train.tgt') as file:
    train_target = file.readlines()
    
with open('./ParaSCI/Data/ParaSCI-ACL/test/test.src') as file:
    test_source = file.readlines()

with open('./ParaSCI/Data/ParaSCI-ACL/test/test.tgt') as file:
    test_target = file.readlines()
    
with open('./ParaSCI/Data/ParaSCI-ACL/val/val.src') as file:
    val_source = file.readlines()

with open('./ParaSCI/Data/ParaSCI-ACL/val/val.tgt') as file:
    val_target = file.readlines()

In [17]:
train_dataset = Dataset.from_dict({
    'train': [{'input': data[0], 'output': data[1]} for data in zip(train_source, train_target)],
})

In [18]:
train_dataset = Dataset.from_dict({
    'train': [{'input': data[0], 'output': data[1]} for data in zip(train_source, train_target)],
})
test_dataset = Dataset.from_dict({
    'test': [{'input': data[0], 'output': data[1]} for data in zip(test_source, test_target)],
})
val_dataset = Dataset.from_dict({
    'val': [{'input': data[0], 'output': data[1]} for data in zip(val_source, val_target)]
})

In [19]:
model_checkpoint = "distilgpt2"

from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [30]:
def tokenize_function(texts):
    return tokenizer([text['input'] for text in list(texts.values())[0]], [text['output'] for text in list(texts.values())[0]])

tokenized_datasets_train = train_dataset.map(tokenize_function, batched=True)
tokenized_datasets_test = test_dataset.map(tokenize_function, batched=True)


  0%|          | 0/29 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

[[1169,
  12665,
  547,
  11241,
  1143,
  290,
  636,
  12,
  1659,
  45862,
  30509,
  351,
  262,
  12067,
  84,
  610,
  74,
  17044,
  299,
  34431,
  2891,
  290,
  336,
  272,
  3841,
  4755,
  21283,
  79,
  764,
  198,
  1640,
  477,
  5050,
  837,
  262,
  12665,
  547,
  11241,
  1143,
  351,
  262,
  12067,
  84,
  17044,
  299,
  34431,
  2891,
  764,
  198],
 [2817,
  263,
  39891,
  2123,
  435,
  837,
  329,
  4554,
  837,
  905,
  326,
  21231,
  39522,
  837,
  290,
  4361,
  2827,
  28456,
  837,
  460,
  307,
  6108,
  422,
  5509,
  48106,
  14599,
  76,
  945,
  764,
  198,
  270,
  373,
  3402,
  416,
  299,
  5702,
  39891,
  2123,
  435,
  326,
  21231,
  39522,
  460,
  635,
  307,
  6840,
  29231,
  329,
  1861,
  14991,
  2569,
  5509,
  48106,
  14599,
  76,
  945,
  764,
  198],
 [11085,
  837,
  479,
  1134,
  22200,
  2123,
  435,
  3088,
  284,
  1630,
  262,
  4129,
  286,
  262,
  6827,
  7560,
  416,
  281,
  2207,
  12342,
  12,
  12501,
  12342,
  

In [39]:
block_size = tokenizer.model_max_length
print(tokenizer.model_max_length)

from transformers import TrainingArguments, Trainer

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)



from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epoch=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)


In [40]:
trainer.train()

ValueError: expected sequence of length 36 at dim 1 (got 34)

In [10]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 233
  Batch size = 8


Perplexity: 1.54


In [11]:
trainer.save_model('./distilgpt2-finetuned-arc')

Saving model checkpoint to ./distilgpt2-finetuned-arc
Configuration saved in ./distilgpt2-finetuned-arc/config.json
Model weights saved in ./distilgpt2-finetuned-arc/pytorch_model.bin
